In [1]:
import pandas as pd
import numpy as np

In [122]:
order_list = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='OrderList')
freight_rates = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='FreightRates')
wh_costs = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='WhCosts')
wh_capacities = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='WhCapacities')
products_per_plant = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='ProductsPerPlant')
vmi_customers = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='VmiCustomers')
plant_ports = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='PlantPorts')


In [47]:
print("\n**************************Order list*****************************************")
print(order_list.describe().T)
print("\n****************************Freight Rates***************************************")
print(freight_rates.describe().T)
print("\n**************************Wh Costs*****************************************")
print(wh_costs.describe().T)
print("\n**************************Wh Capacities*****************************************")
print(wh_capacities.describe().T)
print("\n**************************Products Per Plant*****************************************")
print(products_per_plant.describe().T)
print("\n***************************Vmi Customers****************************************")
print(vmi_customers.describe().T)
print("\n*****************************Plant Ports**************************************")
print(plant_ports.describe().T)
print("*******************************************************************")


**************************Order list*****************************************
                       count                 mean                  min  \
Order ID              9215.0    1447273698.881769         1447125805.7   
Order Date              9215  2013-05-26 00:00:00  2013-05-26 00:00:00   
TPT                   9215.0             1.717743                  0.0   
Ship ahead day count  9215.0             1.852306                  0.0   
Ship Late Day count   9215.0             0.039935                  0.0   
Product ID            9215.0       1680536.058709            1613321.0   
Unit quantity         9215.0          3202.747151                235.0   
Weight                9215.0            19.871688                  0.0   

                                      25%                  50%  \
Order ID                     1447196681.2         1447275684.7   
Order Date            2013-05-26 00:00:00  2013-05-26 00:00:00   
TPT                                   1.0               

In [123]:
# Count unique items in each column for each sheet and check for missing value

# since the order date is the same, let us drop the column
order_list.drop(columns='Order Date', axis=1, inplace=True)

dataframes = {
    "OrderList": orderList,
    "FreightRates": freight_rates,
    "WarehouseCosts": wh_costs,
    "WarehouseCapacities": wh_capacities,
    "ProductsPerPlant": products_per_plant,
    "VMICustomers": vmi_customers,
    "PlantPorts": plant_ports
}

for name, df in dataframes.items():
    print(f"\n{name}")
    print(df.astype('object').describe(include='all').loc['unique', :])
    print(f"\n{name} - Missing Values:")
    print(df.isnull().sum())

print("*******************************************")
print("             Summary Statistics            ")
print("*******************************************")


# Summarize key statistics for categorical columns
for name, df in dataframes.items():
    if not df.select_dtypes(include=['object']).empty:
        print(f"{name} - Statistics for Categorical Data:")
        print(df.select_dtypes(include=['object']).describe())
        print("\n")


OrderList
Order ID                9215.0
Order Date                   1
Origin Port                  3
Carrier                      3
TPT                          5
Service Level                3
Ship ahead day count         5
Ship Late Day count          4
Customer                    46
Product ID                 772
Plant Code                   7
Destination Port             1
Unit quantity             2624
Weight                  5899.0
Name: unique, dtype: object

OrderList - Missing Values:
Order ID                0
Order Date              0
Origin Port             0
Carrier                 0
TPT                     0
Service Level           0
Ship ahead day count    0
Ship Late Day count     0
Customer                0
Product ID              0
Plant Code              0
Destination Port        0
Unit quantity           0
Weight                  0
dtype: int64

FreightRates
Carrier              9
orig_port_cd        10
dest_port_cd         1
minm_wgh_qty     328.0
max_wgh_qty    

In [124]:
# print column names of each data to see whether columns are repeated or need renaming before merging
for name, df in dataframes.items():
    print(f"\n{name} - Columns:")
    print(df.columns)



OrderList - Columns:
Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')

FreightRates - Columns:
Index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'minm_wgh_qty',
       'max_wgh_qty', 'svc_cd', 'minimum cost', 'rate', 'mode_dsc',
       'tpt_day_cnt', 'Carrier type'],
      dtype='object')

WarehouseCosts - Columns:
Index(['WH', 'Cost/unit'], dtype='object')

WarehouseCapacities - Columns:
Index(['Plant ID', 'Daily Capacity '], dtype='object')

ProductsPerPlant - Columns:
Index(['Plant Code', 'Product ID'], dtype='object')

VMICustomers - Columns:
Index(['Plant Code', 'Customers'], dtype='object')

PlantPorts - Columns:
Index(['Plant Code', 'Port'], dtype='object')


In [125]:
# Rename columns so they can be the same across the sheets

freight_rates.rename({'orig_port_cd': 'Origin Port', 'dest_port_cd': 'Destination Port', 'tpt_day_cnt': 'TPT'  }, axis=1, inplace=True)
wh_costs.rename({'WH': 'Plant Code'}, axis=1, inplace=True)
wh_capacities.rename({'Plant ID': 'Plant Code'}, axis=1, inplace=True)

In [126]:
# Merge plant data based on the plant code

from functools import reduce
dframes = [wh_costs, wh_capacities, products_per_plant, vmi_customers.reset_index(drop=True), plant_ports]
merged_plants_data = reduce(lambda  left,right: pd.merge(left,right,on=['Plant Code']), dframes)
merged_plants_data


,Plant Code,Cost/unit,Daily Capacity,Product ID,Customers,Port
0,PLANT02,0.477504,138,1656889,V5555555555555_16,PORT03
1,PLANT02,0.477504,138,1656889,V555555555555555_29,PORT03
2,PLANT02,0.477504,138,1656889,V555555555_3,PORT03
3,PLANT02,0.477504,138,1656889,V55555555555555_8,PORT03
4,PLANT02,0.477504,138,1656889,V55555555_9,PORT03
...,...,...,...,...,...,...
1923,PLANT11,0.555247,332,1690627,V5555555555555555555_54,PORT04
1924,PLANT11,0.555247,332,1690628,V5555555555555555555_54,PORT04
1925,PLANT11,0.555247,332,1690630,V5555555555555555555_54,PORT04
1926,PLANT11,0.555247,332,1690634,V5555555555555555555_54,PORT04


In [130]:
#Merge order and freight data

order_freight_rates = pd.concat((order_list,freight_rates))
print(order_freight_rates.isna().sum())
order_freight_rates.fillna("N/A")


Order ID                1540
Origin Port                0
Carrier                    0
TPT                        0
Service Level           1540
Ship ahead day count    1540
Ship Late Day count     1540
Customer                1540
Product ID              1540
Plant Code              1540
Destination Port           0
Unit quantity           1540
Weight                  1540
minm_wgh_qty            9215
max_wgh_qty             9215
svc_cd                  9215
minimum cost            9215
rate                    9215
mode_dsc                9215
Carrier type            9215
dtype: int64


,Order ID,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight,minm_wgh_qty,max_wgh_qty,svc_cd,minimum cost,rate,mode_dsc,Carrier type
0,1447296446.7,PORT09,V44_3,1,CRF,3.0,0.0,V55555_53,1700106.0,PLANT16,PORT09,808.0,14.3,N/A,N/A,N/A,N/A,N/A,N/A,N/A
1,1447158014.7,PORT09,V44_3,1,CRF,3.0,0.0,V55555_53,1700106.0,PLANT16,PORT09,3188.0,87.94,N/A,N/A,N/A,N/A,N/A,N/A,N/A
2,1447138898.7,PORT09,V44_3,1,CRF,3.0,0.0,V55555_53,1700106.0,PLANT16,PORT09,2331.0,61.2,N/A,N/A,N/A,N/A,N/A,N/A,N/A
3,1447363527.7,PORT09,V44_3,1,CRF,3.0,0.0,V55555_53,1700106.0,PLANT16,PORT09,847.0,16.16,N/A,N/A,N/A,N/A,N/A,N/A,N/A
4,1447363980.7,PORT09,V44_3,1,CRF,3.0,0.0,V55555_53,1700106.0,PLANT16,PORT09,2163.0,52.34,N/A,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,N/A,PORT03,V444_8,0,N/A,N/A,N/A,N/A,N/A,N/A,PORT09,N/A,N/A,0.0,99.99,DTD,29.3392,0.4116,AIR,V88888888_0
1536,N/A,PORT03,V444_8,0,N/A,N/A,N/A,N/A,N/A,N/A,PORT09,N/A,N/A,500.0,1999.99,DTD,29.3392,0.3112,AIR,V88888888_0
1537,N/A,PORT03,V444_8,0,N/A,N/A,N/A,N/A,N/A,N/A,PORT09,N/A,N/A,250.0,499.99,DTD,29.3392,0.4116,AIR,V88888888_0
1538,N/A,PORT03,V444_8,0,N/A,N/A,N/A,N/A,N/A,N/A,PORT09,N/A,N/A,2000.0,99999.99,DTD,29.3392,0.3112,AIR,V88888888_0


### We currently have two aggregated data. 
### 1. merged_plants_data which is a combination of the plants and warehouse data.
### 2. order_freight_rates which is the combination of order list and freight rates.
### We will save both as separate csv files to be used later.

In [132]:
merged_plants_data.to_csv('plants_data.csv', index=False)
order_freight_rates.to_csv('order_freight_rates.csv', index=False)